In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt

from transformer import TransformerConfig
from lm import LM

from data import Dataset

In [31]:
dataset = Dataset()

config = TransformerConfig(d_model=64, n_layers=1, n_heads=8, max_len=dataset.max_len, dropout=0.)
model = LM(config, vocab_size=len(dataset.vocabulaire))
optim = torch.optim.AdamW(model.parameters(), lr=3e-4)

In [32]:
sum(p.numel() for p in model.parameters())

71680

In [22]:
for i in range(10000):
    X, Y = dataset.get_batch('train', 32) # (B, L)
    logits = model(X) # (B, L, vocab_size)

    loss = F.cross_entropy(logits.view(-1, logits.size(-1)), Y.view(-1), ignore_index=dataset.char_to_int['<pad>'])
    optim.zero_grad()
    loss.backward()
    optim.step()
    
    if i%1000==0:
        train_loss = loss.item()

        X, Y = dataset.get_batch('test', 128) # (B, L)
        logits = model(X) # (B, L, vocab_size)
        test_loss = F.cross_entropy(logits.view(-1, logits.size(-1)), Y.view(-1), ignore_index=dataset.char_to_int['<pad>']).item()

        print(f"train loss : {train_loss:.2f} | test loss : {test_loss:.2f}")

tensor(3.8444, grad_fn=<NllLossBackward0>)
tensor(2.1435, grad_fn=<NllLossBackward0>)
tensor(1.9625, grad_fn=<NllLossBackward0>)
tensor(1.7735, grad_fn=<NllLossBackward0>)
tensor(1.9033, grad_fn=<NllLossBackward0>)
tensor(1.7987, grad_fn=<NllLossBackward0>)
tensor(1.6915, grad_fn=<NllLossBackward0>)
tensor(1.7960, grad_fn=<NllLossBackward0>)
tensor(1.7804, grad_fn=<NllLossBackward0>)
tensor(1.8235, grad_fn=<NllLossBackward0>)


In [27]:
X, Y = dataset.get_batch('val', 256) # (B, L)
logits = model(X) # (B, L, vocab_size)

loss = F.cross_entropy(logits.view(-1, logits.size(-1)), Y.view(-1), ignore_index=dataset.char_to_int['<pad>'])
loss

tensor(1.8288, grad_fn=<NllLossBackward0>)

In [28]:
torch.save(model.state_dict(), "transformer_d_model64.pth")

In [35]:
# transformer_d_model16 : loss de 1.85


In [17]:
def sample(self, prompt = "", g = torch.Generator()):
    idx = torch.tensor([dataset.char_to_int[c] for c in prompt], dtype=torch.int32).unsqueeze(0)
    idx = torch.cat([torch.tensor(dataset.char_to_int['<SOS>']).view(1, 1), idx], dim=1)
    next_id = -1

    while next_id != dataset.char_to_int['<EOS>']:
        logits = self(idx) # (1, l, d_model)

        probs = F.softmax(logits[:, -1, :], dim=-1)
        next_id = torch.multinomial(probs, num_samples=1, generator=g).item()
        idx = torch.cat([idx, torch.tensor(next_id).view(1, 1)], dim=1)
        
    return "".join([dataset.int_to_char[p.item()] for p in idx[0, 1:-1]])

In [18]:
for _ in range(10):
    print(sample(model))

 t


IndexError: index out of range in self